# MetroMaps

## Report Exercise 3 - Group Coding Topic a1


## 1. Implementation

### a. Requirements
The first decision was the exact requirements we want to implement and we decided on the following:
1. U-Matrix as Background with selectable color mapping
2. User selectable subset of attributes to show
3. Checkboxes for Snapping of the metro lines and labelling
4. Labelling includes the name of the attribute and the high-/low-points
5. Integrated in the already existing framework

An important part to note is that the given framework already contains an implementation of the Metro Map, but it doesn't include labelling and the background is a component plane in case of a single attribute being selected or nothing in case of multiple attributes being selected.

Since this existing code already implemented the computation of the Metro lines in snapped and not snapped form, we decided to reuse this code and create a new visualization with that.

In the code and in the user interface it is called "MetroMap2" to differntiate it with the already existing implementation. It is integrated in the exact same way as the other already given implementations, which means that there is a controller named `MetroMap2Controller` in the `controls/controllers.py` file and a `MetroMap2` class in the `visualizations/metromap2.py` file. Additionally to also show it in the mainview (via `SOMToolbox._mainview` it is also integrated in the `somtoolbox.py` file.

This allows to use the SOMToolbox code as before with a code snipped like:

In [2]:
from somtoolbox import SOMToolbox
from SOMToolBox_Parse import SOMToolBox_Parse

idata = SOMToolBox_Parse("./datasets/10clusters/10clusters.vec").read_weight_file()
attributes = SOMToolBox_Parse("./datasets/10clusters/10clusters.tv").read_tv_file()
weights = SOMToolBox_Parse("./datasets/10clusters/10clusters.wgt").read_weight_file()

sm = SOMToolbox(weights=weights['arr'], m=10, n=10, dimension=10, input_data=idata['arr'], component_names=attributes)
sm._mainview

Column
    [0] Column
        [0] Column
            [0] Row(margin=5, width=700)
                [0] StaticText(value='<b></b>')
                [1] Select(options=OrderedDict([('Component P...]), value=0)
                [2] Select(options=OrderedDict([('PiYG', ...]), value='jet')
                [3] Button(name='↩')
                [4] Button(name='↪')
                [5] Checkbox(name='interpolation')
        [1] Row
            [0] Column
                [0] HoloViews(DynamicMap)
            [1] Row
                [0] Column
                    [0] Column(margin=5, name='Component Planes', width=300)
                        [0] StaticText(value='<b>Component Planes</b>')
                        [1] IntSlider(end=9, name='comp_1')
                    [1] Str(str)
    [1] Column
        [0] Row()

The visualizations dropdown will show the the option `Metro Map 2.0` which is our implemented solution.

### b. Code
As already mentioned the code of the existing MetroMap implementation was copied and modified to fit our needs. These modifications have been made in the visualization class, but also in the controller.

#### I. Visualization
First the background needed to be changed to show the U-Matrix no matter which components are selected. To achieve this, we used the already provided function to compute the U-Matrix and added it to the `visualizations/metromap2.py` file.

The old metro map visualization computed the background like this:
```py
if len(self._controls.components_int) != 1:
    data = hv.Image(self._main._pipe.data).apply.opts(cmap='Blues', color_levels=self._controls.param.water_level, width=self._main._width, height=self._main._height, xlim=self._main._xlim, ylim=self._main._ylim)
else:
    data = hv.Image(self._digitize(self._controls.components_int[0])).apply.opts(cmap='jet', width=self._main._width, height=self._main._height, xlim=self._main._xlim, ylim=self._main._ylim)
```

This code part was replaced by this:
```py
data = hv.Image(UMatrix(self._main._m, self._main._n, self._main._weights, self._main._dim)) \
    .apply \
    .opts(
    cmap=self._main._maincontrol.param.colormap,
    xlim=self._main._xlim,
    ylim=self._main._ylim,
    width=self._main._width,
    height=self._main._height
)
```
The new code uses the UMatrix to compute the data for the `hv.Image` object. For the new image the limitations and dimensions are set as before, but we also include `cmap=self._main._maincontrol.param.colormap` which makes it possible to react to changes of the selected colormap dropdown in the main view.

For the labelling we decided to split it into two parts. First the labelling of the end points which is done in when also drawing the endpoints themselves.

The code to draw the endpoints:
```py
for i, pts in enumerate(lines):
    c = colors.Category20[20][self._controls.components_int[i]]
    overlay.append(hv.Path(np.array(pts)).opts(color=c, line_width=4))
    overlay.append(hv.Points(np.array(pts)).opts(fill_color=c, color='black', size=15))
```

as extended with:
```py
    if self._controls.labelling:
        end, start = pts[0], pts[-1]
        # endpoint labels
        overlay.append(hv.Text(start[0], start[1], "+", 15))
        overlay.append(hv.Text(end[0], end[1], "-", 15))
```
So when drawing the lines themselves and their endpoints we also draw a `+` or `-` on the endpoints if the `self._controls.labelling` parameter is `True`. This is the case when the checkbox has been checked.

The second part is the labelling of the lines themselves with the corresponding attribute name, which was done by the following code:
```py
for i, pts in enumerate(lines):
    c = colors.Category20[20][self._controls.components_int[i]]
    if self._controls.labelling:
        label = self._main._component_names[self._controls.components_int[i]]
        start = pts[-1]

        # attribute label shadow
        overlay.append(hv.Text(start[0] + 0.028, start[1] - 0.003, label, 12, halign='left').opts(color='black'))
        # attribute label
        overlay.append(hv.Text(start[0] + 0.025, start[1], label, 12, halign='left').opts(color=c))
```
As can bee seen here we again iterate over all the lines and drawn the attribute labels seperately. This has the effect that the labels will always be on top of the metro lines and not obstructed by them.

#### II. Controller
The controller manages the controls that are shown, when selecting a visualization. For the controller of the MetroMap2 visualization, we again used the existing `MetroMapController` class, copied it and modified it to our needs.

The first change was to remove the water level controls, as they are not needed for the U-Matrix visualization we chose.
Secondly, we added a checkbox that can be used to show or hide the labels.
This was done by adding the line 
```py
labelling = param.Boolean(False, label='Show labels')
```
Adding only this line will lead to a checkbox being shown, but nothing will change when selecting it, even with the changes in the visualization class. For this to happen we need to react when the checkbox state is changed, by adding the following lines:
```py
@param.depends("labelling", watch=True)
def _change_labels(self, ):
    self._calculate(False)
```
These lines add a watcher to the labelling parameter and if the checkbox is checked or unchecked it will call the `self._calcuate(False)` method, which causes a repaint without recomputation.

To actually use this controller the line:
```py
self._controls = MetroMapController(self._calculate, self._main._dim, self._main._component_names,name='Metro Map visualization')
```
was changed to
```py
self._controls = MetroMap2Controller(self._calculate, self._main._dim, self._main._component_names,name='Metro Map visualization 2.0')
```
in the `MetroMap2` class.

With this change, when our visualization is selected, then there will be a checkbox to (de-)activate the labelling instead of the water levels.